In [1]:
from twikit import Client, TooManyRequests
import time
from datetime import datetime
import csv
from configparser import ConfigParser
from random import randint
import asyncio

In [7]:
MINIMUM_TWEETS = 100
QUERY = 'martin'

In [3]:
#* login credentials
config = ConfigParser()
config.read('config.ini')
username = config['X']['username']
email = config['X']['email']
password = config['X']['password']
client = Client('en-US')

In [4]:
# Initialize client
#client = Client('en-US')
#client.login(auth_info_1=username, auth_info_2=email, password=password)
#async def main():
#     await client.login(
#         auth_info_1=username,
#         auth_info_2=email,
#         password=password
#     )
# await main()

<coroutine object Client.login at 0x7fdb3d6f2ce0>

In [4]:
with open('tweets.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Tweet_count', 'Username', 'Text', 'Created At', 'Retweets', 'Likes'])

In [5]:
async def get_tweets(tweets):
    if tweets is None:
        #* get tweets
        print(f'{datetime.now()} - Getting tweets...')
        tweets = await client.search_tweet(QUERY, product='Top')
    else:
        wait_time = randint(5, 10)
        print(f'{datetime.now()} - Getting next tweets after {wait_time} seconds ...')
        time.sleep(wait_time)
        tweets = await tweets.next()
    return tweets


In [9]:
async def main():
    tweets = None  # Initialize your tweets variable
    MINIMUM_TWEETS = 100
    tweet_count = 0 
    await client.login(auth_info_1=username,auth_info_2=email,password=password)
   
    # Now you can safely iterate over the tweets
    while tweet_count < MINIMUM_TWEETS:
        try:
            tweets = await get_tweets(tweets)  # Await the tweet fetching coroutine
        except TooManyRequests as e:
            rate_limit_reset = datetime.fromtimestamp(e.rate_limit_reset)
            print(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
            wait_time = rate_limit_reset - datetime.now()
            time.sleep(wait_time.total_seconds())
            continue
    
        if not tweets:
            print(f'{datetime.now()} - No more tweets found')
            break
    
        for tweet in tweets:
            tweet_count += 1
            tweet_data = [tweet_count, tweet.user.name, tweet.text, tweet.created_at, tweet.retweet_count, tweet.favorite_count]
            
            with open('tweets.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(tweet_data)
    
        print(f'{datetime.now()} - Got {tweet_count} tweets')

    print(f'{datetime.now()} - Done! Got {tweet_count} tweets found')


await main()

2024-10-12 17:29:16.773462 - Getting tweets...
2024-10-12 17:29:18.907113 - Got 19 tweets
2024-10-12 17:29:18.907222 - Getting next tweets after 9 seconds ...
2024-10-12 17:29:30.517202 - Got 39 tweets
2024-10-12 17:29:30.517293 - Getting next tweets after 9 seconds ...
2024-10-12 17:29:41.787090 - Got 59 tweets
2024-10-12 17:29:41.787194 - Getting next tweets after 6 seconds ...
2024-10-12 17:29:50.666493 - Got 79 tweets
2024-10-12 17:29:50.666603 - Getting next tweets after 10 seconds ...
2024-10-12 17:30:03.290589 - Got 99 tweets
2024-10-12 17:30:03.290707 - Getting next tweets after 10 seconds ...
2024-10-12 17:30:15.740141 - Got 119 tweets
2024-10-12 17:30:15.740236 - Done! Got 119 tweets found
